# Executive Healthcare KPIs

## Objective
Compute high-impact KPIs related to appointments, revenue, payments,
and patient value to support healthcare management decision-making.


In [1]:
import pandas as pd
import os


In [2]:
BASE_PATH = "C:/Users/ra481/OneDrive/Desktop/healthcare-analytics"
DATA_PATH = f"{BASE_PATH}/Data/Raw"
REPORTS_PATH = f"{BASE_PATH}/reports"

os.makedirs(REPORTS_PATH, exist_ok=True)


In [3]:
appointments_df = pd.read_csv(f"{DATA_PATH}/appointments.csv")
billing_df = pd.read_csv(f"{DATA_PATH}/billing.csv")


In [4]:
appointments_df["appointment_date"] = pd.to_datetime(
    appointments_df["appointment_date"]
)

billing_df["bill_date"] = pd.to_datetime(billing_df["bill_date"])


In [5]:
total_appointments = len(appointments_df)

cancellation_rate = (
    len(appointments_df[appointments_df["status"] == "Cancelled"])
    / total_appointments * 100
)

completion_rate = (
    len(appointments_df[appointments_df["status"] == "Completed"])
    / total_appointments * 100
)

round(cancellation_rate, 2), round(completion_rate, 2)


(25.5, 23.0)

In [6]:
average_bill_amount = billing_df["amount"].mean()
round(average_bill_amount, 2)


np.float64(2756.25)

In [7]:
total_revenue = billing_df["amount"].sum()

unpaid_revenue = billing_df[
    billing_df["payment_status"] != "Paid"
]["amount"].sum()

revenue_leakage_rate = unpaid_revenue / total_revenue * 100
round(revenue_leakage_rate, 2)


np.float64(68.54)

In [8]:
payment_success = (
    billing_df
    .groupby("payment_method")["payment_status"]
    .apply(lambda x: (x == "Paid").mean() * 100)
    .reset_index(name="success_rate_percent")
)

payment_success


,payment_method,success_rate_percent
0,Cash,32.786885
1,Credit Card,32.000000
2,Insurance,31.250000


In [9]:
patient_visit_counts = (
    appointments_df
    .groupby("patient_id")
    .size()
)

retained_patients = (patient_visit_counts > 1).sum()
total_patients = patient_visit_counts.shape[0]

patient_retention_rate = retained_patients / total_patients * 100
round(patient_retention_rate, 2)


np.float64(100.0)

In [10]:
revenue_per_patient = total_revenue / total_patients
round(revenue_per_patient, 2)


np.float64(11484.37)

In [11]:
completed_appointments = len(
    appointments_df[appointments_df["status"] == "Completed"]
)

revenue_per_appointment = total_revenue / completed_appointments
round(revenue_per_appointment, 2)


np.float64(11983.69)

In [12]:
executive_kpis = pd.DataFrame({
    "KPI": [
        "Cancellation Rate (%)",
        "Completion Rate (%)",
        "Average Bill Amount",
        "Revenue Leakage Rate (%)",
        "Patient Retention Rate (%)",
        "Revenue per Patient",
        "Revenue per Appointment"
    ],
    "Value": [
        round(cancellation_rate, 2),
        round(completion_rate, 2),
        round(average_bill_amount, 2),
        round(revenue_leakage_rate, 2),
        round(patient_retention_rate, 2),
        round(revenue_per_patient, 2),
        round(revenue_per_appointment, 2)
    ]
})

executive_kpis


,KPI,Value
0,Cancellation Rate (%),25.50
1,Completion Rate (%),23.00
2,Average Bill Amount,2756.25
3,Revenue Leakage Rate (%),68.54
4,Patient Retention Rate (%),100.00
5,Revenue per Patient,11484.37
6,Revenue per Appointment,11983.69


In [13]:
executive_kpis.to_csv(
    f"{REPORTS_PATH}/executive_kpis.csv",
    index=False
)


In [14]:
os.path.exists(f"{REPORTS_PATH}/executive_kpis.csv")


True

## Key Executive Insights
- Identified operational inefficiencies through cancellation and completion rates
- Quantified revenue leakage from unpaid bills
- Evaluated payment channel reliability
- Measured patient retention and revenue contribution
